In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import csv
import sys
import math
import cv2
import scipy
import pickle
import librosa
import matplotlib
import numpy as np
import librosa.display
import IPython.display as ipd
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras import models
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Conv1D, Flatten,Dropout,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import SGD

# split data into test and train
def split_data_test_train(prog,nonprog) :
    ind1 = math.floor(len(nonprog)/3)
    ind2 = math.floor(len(prog)/3)
    x_train1 = nonprog[0:ind1]
    y_train1 = np.ones((ind1,1 ))
    x_test1 = nonprog[ind1:]
    y_test1 = np.ones((len(nonprog) - ind1,1) )
    print(len(x_test1) )
    print(len(y_test1) )

    x_train2 = prog[0:ind2]
    y_train2 = np.zeros( ( ind2,1 ))
    x_test2 = prog[ind2:]
    y_test2 = np.zeros( (len(prog) - ind2,1 ))

    print(len(x_test2) )
    print(len(y_test2) )


    x_test = x_train1 + x_train2
    x_train = x_test1 + x_test2
    y_test = np.concatenate((y_train1, y_train2), axis=0)
    y_train = np.concatenate((y_test1, y_test2), axis=0)
    print("x_train ",len(x_train)  )
    print("x-test ",len(x_test)  )
    print("y_train ",len(y_train)  )
    print("y_test ",len(y_test)  )
    return [x_train,x_test,y_train,y_test]


# Get train data from csv file
x_train = []
y_train = []
prog = []

nonprog = []

data = pd.read_csv('/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/training_features_10_secs.csv')
# print(data.head())
data = data.drop(['filename'],axis=1)
data = np.asarray(data)
X = data[:,6:]
y = data[:,0]
y[y=="prog"] = 0
y[y=="non_prog"] = 1

x_train, x_test, y_train, y_test = train_test_split( X, y, test_size=0.5, random_state=42)

X = np.reshape(X, (X.shape[0],1,X.shape[1]))


print("X_train shape ",x_train.shape)
print("y_train shape ",y_train.shape)
print(x_train)
print(y_train)
print("X_test shape ",x_test.shape)
print("y_test shape ",y_test.shape)
# print(x_test)
# print(y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# y = to_categorical(y)

# Build model using LSTM
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)  # Example: Convert to float32
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)    # Example: Convert to int32
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)  # Example: Convert to float32
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)    # Example: Convert to int32
x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)
input_shape = (1,X.shape[2])
kfold = KFold(n_splits=3, shuffle=True, random_state=45)
cvscores = []
batch_size = 35
nb_epochs = 100
opt = tf.keras.optimizers.legacy.Adam()

count = 1
for train, test in kfold.split(X, y):
  # Create model
    model = Sequential()
    model.add(LSTM(units=64, dropout=0.01, recurrent_dropout=0.35, return_sequences=True,input_shape=input_shape) )
    model.add(LSTM(units=32, dropout=0.01, recurrent_dropout=0.35, return_sequences=False))
    model.add(Dense(units=2, activation='sigmoid'))

    print("Compiling ...")
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()

    print("Training ...")
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epochs,validation_split=0.33)
    score, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
    print("Test loss:  ", score)
    print("Test accuracy:  ", accuracy)

    # Summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("/content/drive/MyDrive/Music-Genre-Classification-master"+str(count)+"accuracy")
    plt.close()

    # Summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("/content/drive/MyDrive/Music-Genre-Classification-master"+str(count))
    plt.close()

    count += 1
model.save('Model.h5')

X_train shape  (9805, 21)
y_train shape  (9805,)
[[10.6264210892445 -61.96282196044922 119.2556610107422 ...
  3.4255645275115967 -3.8072726726531982 -0.095552310347557]
 [41.32490839457791 -425.28759765625 126.89898681640624 ...
  -1.933012008666992 -0.6390572190284729 4.548562526702881]
 [9.163107979146176 -102.97955322265624 166.9093475341797 ...
  -1.167567253112793 -0.810927152633667 0.4991808533668518]
 ...
 [21.718575865449434 -104.3449478149414 127.9443130493164 ...
  11.56142807006836 2.412437677383423 5.155041217803955]
 [42.165576585869495 -101.73089599609376 173.440185546875 ...
  -0.0102514987811446 -0.805259108543396 -3.790855407714844]
 [36.81000050649021 -279.3218688964844 143.85426330566406 ...
  -3.207831382751465 -3.1452958583831787 -5.238548755645752]]
[0 1 0 ... 0 0 1]
X_test shape  (9806, 21)
y_test shape  (9806,)
Compiling ...
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape         

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn.metrics import confusion_matrix

# Predict probabilities for test set
y_pred_probs = model.predict(x_test)
# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
# Convert one-hot encoded labels back to single column
y_true = np.argmax(y_test, axis=1)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


307/307 [==============================] - 1s 2ms/step
Confusion Matrix:
[[4441  608]
 [ 464 4293]]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
# Calculate confusion matrix
confusion_matrix = confusion_matrix(y_true, y_pred)
# Define class labels for better visualization
class_names = ["Prog", "Non-Prog"]

# Create DataFrame for confusion matrix
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names)

# Plot confusion matrix using seaborn heatmap
plt.figure(figsize=(7, 4))
sn.heatmap(df_cm, annot=True, fmt='d', cmap='Blues', cbar=False,
           annot_kws={"size": 16}, linewidths=0.5, linecolor='black')

# Customize plot
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(np.arange(len(class_names)) + 0.5, class_names, rotation=45)
plt.yticks(np.arange(len(class_names)) + 0.5, class_names, rotation=0)

# Display plot
plt.tight_layout()
display(plt.show())

None

In [ ]:
print(y_true)

[1 1 0 ... 0 1 0]


In [ ]:
# Calculate misclassification error
predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)
misclassified_indices = np.where(predicted_labels != true_labels)[0]
print("Misclassified indices:", misclassified_indices)
misclassified_samples = x_test[misclassified_indices]
#print(misclassified_samples)
misclassification_error = np.sum(predicted_labels != true_labels) / len(true_labels)
print("Misclassification error: ", misclassification_error)


80/80 [==============================] - 0s 5ms/step
Misclassified indices: [  34   36   42   60   66   78   92   93   94   97   98  111  112  115
  130  135  136  150  162  164  166  172  174  177  178  193  195  196
  199  214  222  224  245  247  251  266  267  275  277  278  279  299
  315  316  322  349  362  364  410  419  424  433  435  436  440  469
  486  487  488  496  497  499  509  524  526  551  553  555  556  563
  565  567  572  587  593  599  602  606  608  609  614  652  667  677
  679  680  683  686  696  706  707  713  730  732  756  770  772  774
  775  776  799  800  811  824  828  832  833  834  837  840  843  848
  854  859  867  868  881  892  896  904  906  909  923  925  958  959
  963  967  973  978  985 1009 1011 1012 1015 1017 1022 1030 1039 1049
 1051 1054 1055 1057 1058 1069 1072 1075 1097 1098 1102 1106 1107 1108
 1111 1114 1116 1119 1122 1124 1127 1130 1151 1156 1164 1165 1174 1182
 1185 1188 1191 1195 1202 1205 1209 1214 1224 1225 1231 1238 1247 1254
 

In [ ]:
 import os
import csv
import sys
import math
import cv2
import scipy
import pickle
import librosa
import matplotlib
import numpy as np
import librosa.display
import IPython.display as ipd
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras import models
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Conv1D, Flatten,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import seaborn as sn
import random

prog = []
nonprog = []
total_files = 0
total_prog = 0
total_non_prog = 0

# find total files
# for root, dirs, files in os.walk("/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/Test_set/Progressive Rock Songs", topdown=False):
#    for name in files:
#       filename = os.path.join(root, name)
#       if filename.find(".mp3") != -1 or filename.find(".wav") != -1 or filename.find(".flac") != -1 or filename.find(".avi") != -1 or filename.find(".m4a") != -1 or filename.find(".ogg") != -1 :
#           total_files = total_files + 1
#           total_prog += 1

# for root, dirs, files in os.walk("/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/Test_set/Not_Progressive_Rock", topdown=False):
#    for name in files:
#       filename = os.path.join(root, name)
#       if filename.find(".mp3") != -1 or filename.find(".wav") != -1 or filename.find(".flac") != -1 or filename.find(".avi") != -1 or filename.find(".m4a") != -1 or filename.find(".ogg") != -1 :
#           total_files = total_files + 1
#           total_non_prog += 1

for root, dirs, files in os.walk("/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/Test_set/Other", topdown=False):
   for name in files:
      filename = os.path.join(root, name)
      if filename.find(".mp3") != -1 or filename.find(".wav") != -1 or filename.find(".flac") != -1 or filename.find(".avi") != -1 or filename.find(".m4a") != -1 or filename.find(".ogg") != -1 :
          total_files = total_files + 1
          total_prog += 1


# --------Extract data from csv file--------------------------------------

data = pd.read_csv('/content/drive/MyDrive/Music-Genre-Classification-master-20240427T160246Z-001/Music-Genre-Classification-master/test_djent_features.csv')
data.head()
new_files = data['filename']
new_files = list(new_files)
for i in range(len(new_files)) :
    new_files[i] = (new_files[i].split("chunk") )[0]


data = data.drop(['filename'],axis=1)
data = np.asarray(data)

X = data[:,6:]
y = data[:,0]
y[y=="djent"] = 0


X = np.reshape(X, (X.shape[0],1, X.shape[1]))
X = X.astype(np.float32)
print("X_train shape ",X.shape)
y_test = y
y_test = to_categorical(y_test)


model = load_model('Model.h5')

print("\nTesting ...")

# Predict using saved model

predictions = model.predict(X)
print(predictions)
row,col = predictions.shape
count = 0
for i in range(row) :
    true_val = np.argmax(y_test[i])
    predicted_val = np.argmax(predictions[i])
    # print("true ",true_val,"    got ",predicted_val)
    if true_val == predicted_val :
       count += 1
print("Acc ",count," / ",row)
print("Total files ",total_files)


new_true_ind = []
new_predicted_ind = []
i = 0

ind = []
start = 0
for i in range(len(new_files)) :
    if i == 0 :
        start = 0
    elif i == len(new_files) - 1 :
        if new_files[i] == new_files[i-1] :
            ind.append([new_files[i],start,len(new_files)-1])
        else :
            ind.append([new_files[i],len(new_files)-1,len(new_files)-1])

    else :
         if new_files[i] != new_files[i-1] :
             end = i-1
             ind.append([new_files[i-1],start,end])
             start = i

print("indices calculated")

for i in range(row) :
    new_predicted_ind.append(np.argmax(predictions[i]))
    new_true_ind.append(np.argmax(y_test[i]))

# Find majority result for song chunks

# def calc_majority(indices,predicted,true) :
#     name = indices[0]
#     start = indices[1]
#     end = indices[2]
#     new_predicted = predicted[start:end+1]
#     new_true = true[start:end+1]
#     maj_pred = 0
#     maj_true = 0
#     for i in range(len(new_predicted)) :
#         if new_predicted[i] == 0 :
#             maj_pred += 1
#     if maj_pred <= 2*len(new_predicted)/3  :
#        ret_pred_val = 1
#     else:
#         ret_pred_val=0

#     if name.find("nonprog") != -1 :
#         ret_true_val = 1
#     else :
#         ret_true_val = 0
#     return [ret_pred_val,ret_true_val]

def calc_majority(indices, predicted, true):
    name = indices[0]
    start = indices[1]
    end = indices[2]
    new_predicted = predicted[start:end+1]
    new_true = true[start:end+1]
    maj_pred = 0
    for pred in new_predicted:
        if pred == 0:
            maj_pred += 1
    ret_pred_val = 0 if maj_pred > len(new_predicted)/2 else 1

    ret_true_val = 1 if name.find("nonprog") != -1 else 0
    return name, ret_pred_val, ret_true_val  # Return name as well



majority_pred = []
majority_true = []

print("len ind",len(ind))

# Find accuracy for all chunks by checking majority output
failed_predictions = []

for i in range(len(ind)):
    name, ret_pred_val, ret_true_val = calc_majority(ind[i], new_predicted_ind, new_true_ind)
    majority_pred.append(ret_pred_val)
    majority_true.append(ret_true_val)
    if ret_pred_val != ret_true_val:
        failed_predictions.append((name, ret_true_val, ret_pred_val))

count = 0
print("len maj",len(majority_pred))
for i in range(len(majority_pred)) :
    #print("true ",majority_true[i],"     predicted",majority_pred[i])
    if majority_pred[i] == majority_true[i] :
        count += 1
# Assuming majority_true and majority_pred are lists of equal length
assert len(majority_true) == len(majority_pred)


# Open a new file to write the data
with open('majority_predictions.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Song Name', 'True', 'Predicted'])  # Adjust header to include song names

    # Write only failed predictions
    for item in failed_predictions:
        writer.writerow(item)  # Write each entry





print("Accuracy ",count," / ",total_files, (count/total_files) *100," %" )




X_train shape  (204, 1, 21)



Testing ...
7/7 [==============================] - 1s 5ms/step
[[0.30809194 0.7988365 ]
 [0.07577261 0.9732224 ]
 [0.09193101 0.96643186]
 [0.03911426 0.98762184]
 [0.18380757 0.9061602 ]
 [0.2370923  0.8681174 ]
 [0.20045313 0.89843   ]
 [0.19328569 0.89832085]
 [0.00650749 0.998637  ]
 [0.00768399 0.9984982 ]
 [0.01018575 0.99776155]
 [0.00863327 0.9981464 ]
 [0.00885446 0.99817276]
 [0.01037316 0.9977052 ]
 [0.0082312  0.99836427]
 [0.00885058 0.99820304]
 [0.01488311 0.99657995]
 [0.01069165 0.9975984 ]
 [0.00275306 0.9995107 ]
 [0.00146106 0.99978167]
 [0.00172097 0.99974334]
 [0.0012258  0.99982893]
 [0.00140402 0.9997861 ]
 [0.00137119 0.99980557]
 [0.00279197 0.9995219 ]
 [0.00264166 0.9995742 ]
 [0.00266631 0.9995678 ]
 [0.0036607  0.9994494 ]
 [0.00314038 0.9994379 ]
 [0.37107325 0.62925255]
 [0.24418847 0.79746723]
 [0.191458   0.83901066]
 [0.15997504 0.8764749 ]
 [0.12886089 0.90917647]
 [0.20800757 0.82284236]
 [0.22725032 0.8112729 ]
 [0.41001076 0.5804669 ]
 [0.1853448

In [ ]:
majority_pred

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
import sys
import numpy

numpy.set_printoptions(threshold=sys.maxsize)
print(predicted_val)

1


In [ ]:
misclassification_count = 0

# Find misclassification count
for true_val, pred_val in zip(majority_true, majority_pred):
    if true_val != pred_val:
        misclassification_count += 1

# Calculate misclassification error
misclassification_error = misclassification_count / total_files

print("Misclassification Error: {} / {} ({:.2%})".format(misclassification_count, total_files, misclassification_error))


Misclassification Error: 79 / 266 (29.70%)


In [ ]:
majority_true

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
majority_pred

[1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,


In [ ]:
from sklearn.metrics import confusion_matrix
# Calculate confusion matrix
confusion_matrix = confusion_matrix(majority_true, majority_pred)
# Define class labels for better visualization
class_names = ["Prog", "Non-Prog"]

# Create DataFrame for confusion matrix
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names)

# Plot confusion matrix using seaborn heatmap
plt.figure(figsize=(10, 7))
sn.heatmap(df_cm, annot=True, fmt='d', cmap='Blues', cbar=False,
           annot_kws={"size": 16}, linewidths=0.5, linecolor='black')

# Customize plot
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(np.arange(len(class_names)) + 0.5, class_names, rotation=45)
plt.yticks(np.arange(len(class_names)) + 0.5, class_names, rotation=0)

# Display plot
plt.tight_layout()
display(plt.show())


None

In [ ]:
confusion_matrix = confusion_matrix(majority_true, majority_pred )
print(len(majority_true)," ",len(majority_pred))
df_cm = pd.DataFrame(confusion_matrix, index = [i for i in ["True prog songs","True nonprog songs"]],
                  columns = [i for i in ["Predicted prog songs","Predicted nonprog songs"]])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.title('Total prog = %i, Total nonprog = %i' %(total_prog,total_non_prog) )
df_cm.plot()
plt.show()

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

# Assuming you have defined confusion_matrix, majority_true, majority_pred, total_prog, and total_non_prog correctly

# Check if confusion_matrix is a function or variable
# If it's a variable, ensure it contains the confusion matrix data
# If it's a function, make sure it's properly defined and called
print("Shape of confusion matrix:", confusion_matrix.shape)

# Check if total_prog and total_non_prog are defined and have correct values
print("Total prog:", total_prog)
print("Total nonprog:", total_non_prog)

# Create DataFrame for confusion matrix
df_cm = pd.DataFrame(confusion_matrix, index=["True prog songs", "True nonprog songs"],
                     columns=["Predicted prog songs", "Predicted nonprog songs"])

# Create heatmap
plt.figure(figsize=(10, 7))
sn.heatmap(df_cm, cmap=plt.cm.Blues)
plt.title('Total prog = %i, Total nonprog = %i' % (total_prog, total_non_prog))
plt.show()


Shape of confusion matrix: (2, 2)
Total prog: 136
Total nonprog: 130
